# Overview

This notebook demonstrates how to use Vertex AI Prediction with Feature Store integration. This integration enables feature values to be fetched directly from Feature Store during a `predict` request, instead of having to specify feature values explicitly in the predict request.

**This is an Experimental release**, covered by the Pre-GA Offerings Terms of your Google Cloud Platform [Terms of Service](https://cloud.google.com/terms).

Experimental features are focused on validating a prototype and are not guaranteed to be released. They are not intended for production use or covered by any SLA, support obligation, or deprecation policy and might be subject to backward-incompatible changes. They also may not be at the same reliability standards as a GA product.

**Kindly drop us a note before you run any scale tests.**

**Do not hesitate to contact vertexai-prediction-preview-feedback@google.com if you have any questions or run into any issues.**

The projects need to be allowlisted to access this feature. If you are interested, please send an email to vertexai-prediction-preview-feedback@google.com with your project numbers OR project ids.

The dataset and model used in this notebook is based on this [Prediction and Feature Store Online Serving notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/feature_store/mobile_gaming/mobile_gaming_feature_store.ipynb) and [this blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml).

### Dataset

The dataset is the public sample export data from an actual mobile game app called "Flood It!" (Android, iOS).

### Model

The model used in this notebook is based on [this blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml). The model predicts user churn for game developers. The raw data contains the following categories of information:

- identity - unique player identity numbers.
- demographic features - information about the player, such as the geographic region in which a player is located.
- behavioral features - counts of the number of times a player has triggered certain game events, such as reaching a new level.
- churn propensity - this is the label or target feature, it provides an estimated probability that this player will churn, i.e. stop being an active player.

The blog article referenced above explains how to use BigQuery to store the raw data, pre-process the data for machine learning, and train the corresponding model. Because this notebook focuses on Vertex Prediction, we're going to reuse a pre-trained version of this model, which can be downloaded from `gs://mco-mm/churn`.

### Objective

The goal is to:
- Set up a Feature Store with the mobile game users demographic and behavioral data.
- Learn how to write a feature fetch config for Vertex Prediction.
- Deploy a Feature Store integrated model to Vertex Prediction using the pretrained churn model.
- Send a `predict` request to the deployed model that fetches feature values from Feature Store in real time.
- Learn how to debug a Feature Store integrated model using a identity model.

# Basic set up

## Install packages

In [ ]:
! pip install google-cloud-aiplatform

## Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Set up project

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

In [ ]:
! gcloud config set project $PROJECT_ID

Set the region you choose to use.

In [ ]:
REGION = ""

if REGION == "" or REGION is None:
    REGION = "us-central1"

## Timestamp

This timestamp will be used to prevent resource name collisions.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

## Model Artifacts Bucket Setup

Create a bucket and copy the pretrained model to it.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-mobile-gaming-model-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

Only if your bucket doesn't already exist: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

In [ ]:
! gsutil ls -al $BUCKET_URI

Copy the trained model to your bucket

In [ ]:
ARTIFACT_URI = BUCKET_URI + "/model"

In [ ]:
! gsutil cp -r gs://mco-mm/churn/* $BUCKET_URI/model

# Feature Store Setup

First let's set up a Feature Store to host the data.

## Import Vertex Libraries

In [ ]:
# Vertex AI and its Feature Store
from google.cloud import aiplatform
from google.cloud.aiplatform import Featurestore

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## Create Feature Store

In [ ]:
FEATURESTORE_ID = "mobile_gaming_" + TIMESTAMP  # @param {type:"string"}

# Vertex AI Feature store
ONLINE_STORE_NODES_COUNT = 5
DEMOGRAPHIC_ENTITY_ID = "demographic"
BEHAVIOR_ENTITY_ID = "behavior"
FEATURE_TIME = "timestamp"
ENTITY_ID_FIELD = "user_pseudo_id"

We will use the exported sample data of the dataset used in this [this blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)

In [ ]:
try:
    mobile_gaming_feature_store = Featurestore.create(
        featurestore_id=FEATURESTORE_ID,
        online_store_fixed_node_count=ONLINE_STORE_NODES_COUNT,
        sync=True,
    )
except RuntimeError as error:
    print(error)
else:
    FEATURESTORE_RESOURCE_NAME = mobile_gaming_feature_store.resource_name
    print(f"Feature store created: {FEATURESTORE_RESOURCE_NAME}")

### Create Entities

In [ ]:
try:
    demographic_entity_type = mobile_gaming_feature_store.create_entity_type(
        entity_type_id=DEMOGRAPHIC_ENTITY_ID,
        description="User demographic Entity",
        sync=True,
    )
except RuntimeError as error:
    print(error)
else:
    DEMOGRAPHIC_ENTITY_RESOURCE_NAME = demographic_entity_type.resource_name
    print("Entity type name is", DEMOGRAPHIC_ENTITY_RESOURCE_NAME)

In [ ]:
try:
    behavior_entity_type = mobile_gaming_feature_store.create_entity_type(
        entity_type_id=BEHAVIOR_ENTITY_ID, description="User behavior Entity", sync=True
    )
except RuntimeError as error:
    print(error)
else:
    BEHAVIOR_ENTITY_RESOURCE_NAME = behavior_entity_type.resource_name
    print("Entity type name is", BEHAVIOR_ENTITY_RESOURCE_NAME)

### Create Features

#### Feature Config

In [ ]:
demographic_feature_configs = {
    "country": {
        "value_type": "STRING",
        "description": "The country of customer",
        "labels": {"status": "passed"},
    },
    "operating_system": {
        "value_type": "STRING",
        "description": "The operating system of device",
        "labels": {"status": "passed"},
    },
    "language": {
        "value_type": "STRING",
        "description": "The language of device",
        "labels": {"status": "passed"},
    },
    "user_pseudo_id": {
        "value_type": "STRING",
        "description": "User pseudo id",
        "labels": {"status": "passed"},
    },
}

behavior_feature_configs = {
    "cnt_user_engagement": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement level",
        "labels": {"status": "passed"},
    },
    "cnt_level_start_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with start level",
        "labels": {"status": "passed"},
    },
    "cnt_level_end_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with end level",
        "labels": {"status": "passed"},
    },
    "cnt_level_complete_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with complete status",
        "labels": {"status": "passed"},
    },
    "cnt_level_reset_quickplay": {
        "value_type": "DOUBLE",
        "description": "A variable of user engagement with reset status",
        "labels": {"status": "passed"},
    },
    "cnt_post_score": {
        "value_type": "DOUBLE",
        "description": "A variable of user score",
        "labels": {"status": "passed"},
    },
    "cnt_spend_virtual_currency": {
        "value_type": "DOUBLE",
        "description": "A variable of user virtual amount",
        "labels": {"status": "passed"},
    },
    "cnt_ad_reward": {
        "value_type": "DOUBLE",
        "description": "A variable of user reward",
        "labels": {"status": "passed"},
    },
    "cnt_challenge_a_friend": {
        "value_type": "DOUBLE",
        "description": "A variable of user challenges with friends",
        "labels": {"status": "passed"},
    },
    "cnt_completed_5_levels": {
        "value_type": "DOUBLE",
        "description": "A variable of user level 5 completed",
        "labels": {"status": "passed"},
    },
    "cnt_use_extra_steps": {
        "value_type": "DOUBLE",
        "description": "A variable of user extra steps",
        "labels": {"status": "passed"},
    },
    "month": {
        "value_type": "INT64",
        "description": "First touch month",
        "labels": {"status": "passed"},
    },
    "julianday": {
        "value_type": "INT64",
        "description": "First touch julian day",
        "labels": {"status": "passed"},
    },
    "dayofweek": {
        "value_type": "INT64",
        "description": "First touch day of week",
        "labels": {"status": "passed"},
    },
}

#### Create features using `batch_create_features` method

In [ ]:
try:
    demographic_entity_type.batch_create_features(
        feature_configs=demographic_feature_configs, sync=True
    )
except RuntimeError as error:
    print(error)
else:
    for feature in demographic_entity_type.list_features():
        print("")
        print(f"The resource name of {feature.name} feature is", feature.resource_name)

In [ ]:
try:
    behavior_entity_type.batch_create_features(
        feature_configs=behavior_feature_configs, sync=True
    )
except RuntimeError as error:
    print(error)
else:
    for feature in behavior_entity_type.list_features():
        print("")
        print(f"The resource name of {feature.name} feature is", feature.resource_name)

### Ingest features 

First copy the data to your bucket.

In [ ]:
SOURCE_URI = BUCKET_URI + "/feature_store_source/mobile_gaming_dataset.csv"

In [ ]:
! gsutil cp -r gs://featurestore_prediction_integration/data/mobile_gaming_dataset.csv $SOURCE_URI

In [ ]:
DEMOGRAPHIC_FEATURES_IDS = [
    feature.name for feature in demographic_entity_type.list_features()
]

In [ ]:
# This may take several minutes to run

try:
    demographic_entity_type.ingest_from_gcs(
        feature_ids=DEMOGRAPHIC_FEATURES_IDS,
        feature_time=FEATURE_TIME,
        gcs_source_uris=SOURCE_URI,
        gcs_source_type="csv",
        entity_id_field=ENTITY_ID_FIELD,
        disable_online_serving=False,
        worker_count=10,
        sync=True,
    )
except RuntimeError as error:
    print(error)

In [ ]:
BEHAVIOR_FEATURES_IDS = [
    feature.name for feature in behavior_entity_type.list_features()
]

In [ ]:
# This may take several minutes to run

try:
    behavior_entity_type.ingest_from_gcs(
        feature_ids=BEHAVIOR_FEATURES_IDS,
        feature_time=FEATURE_TIME,
        gcs_source_uris=SOURCE_URI,
        gcs_source_type="csv",
        entity_id_field=ENTITY_ID_FIELD,
        disable_online_serving=False,
        worker_count=10,
        sync=True,
    )
except RuntimeError as error:
    print(error)

Now that we have our Feature Store set up, let's try fetching features for a sample entity.

In [ ]:
demographic_entity_type.read(entity_ids="AB0F2EE5F9F401763BE1E9FA55410312")

In [ ]:
behavior_entity_type.read(entity_ids="AB0F2EE5F9F401763BE1E9FA55410312")

# Feature Fetch Config File

To enable feature fetching for model deployment, the file `prediction_featurestore_fetch_config.yaml` must be included in the `artifacts_uri` path. 

The purpose of this yaml file is to specify the following information.
- Which **feature store** (location, project ID, featurestore ID) to fetch features from.
- The **feature names** (i.e. feature IDs) corresponding to the features to be fetched.
- How to **construct the predict request** from the user provided predict request and fetched features.
- At Prediction Time, which **entities** to fetch features from.

The yaml file must conform to the following proto.

## Feature fetch config proto:

```protobuf
message FeatureFetchConfig {
  // The format of the internal prediction request auto-created after features
  // are fetched. Prediction currently supports XGBoost, TensorFlow,
  // scikit-learn, Pytorch, and custom containers. Among these frameworks, 
  // XGBoost supports array input format only (i.e. input features are in the 
  // form of an array), whereas the other three frameworks can allow both
  // dictionary format and array format inputs.
  ModelInputFormat model_input_format = 3;
  enum ModelInputFormat {
    MODEL_INPUT_FORMAT_UNSPECIFIED = 0;
    ARRAY = 1;
    DICT = 2;
  }

  message FeatureSource {
    // Required. Fully-qualified-name of the EntityType, e.g.:
    // "projects/acme/locations/us-central1/featurestores/fs/entityTypes/movies"
    string entity_type = 2;

    // Required. Specifies the field holding the entityId to be fetched for this
    // sources. Currently, input request is a (JSON) dictionary, it simply
    // corresponds to the dictionary entry with this key.
    string entity_id_field = 3;
  }

  // Feature Source definitions. Key of this map is used to refer to the source
  // in the Feature settings. Key cannot be empty.
  map<string, FeatureSource> feature_sources = 5;

  // Specifying details of the prediction input
  repeated Feature features = 4;
  message Feature {
    // Obsolete fields.
    reserved 1, 2;

    // When internal_request_format = DICT, this name is used
    // for the Internal Prediction Request as the key to the feature value.
    // In the FeatureFetchConfig, a pass-through feature can be represented by
    // a Feature message with just a name.
    string name = 3;

    message FetchFrom {
      // Required. Name of the feature source to fetch feature values from.
      // Leave blank if feature values is from the original request
      // (passthrough).
      string source_name = 1;

      // Required. The Feature ID to be fetched. This should be a valid feature
      // Id for the entityType corresponding to the featureSource above. Must be
      // provided if the featureSource is used.
      string feature_id = 2;
    }
    // Define the source to fetch feature value. Note that if the request sent
    // by the user already provides a value for this feature, that value will be
    // used as is (i.e this setting will be ignored and this feature will be
    // treated as passthrough for that request) If this field is not set, the
    // feature value is always passthrough from the original request.
    FetchFrom fetch_from = 4;
  }
}
```

## Write the Feature Fetch Config

Note the following
- We use `modelInputFormat: DICT` since the model expects a dictionary input
- `user_pseudo_id` is a pass-through feature. The key value pair in the user's predict request is passed directly to the model server.
- All other `features` (with non empty `fetchFrom`) will be fetched from feature store at predict time.

In [ ]:
FEATURE_FETCH_CONFIG_TEMPLATE = """modelInputFormat: DICT
featureSources:
  user:
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/demographic
    entityIdField: demographic
  behavior:
    entityType: projects/{PROJECT_NUMBER}/locations/{REGION}/featurestores/{FEATURESTORE_ID}/entityTypes/behavior
    entityIdField: behavior
features:
- name: user_pseudo_id
- name: country
  fetchFrom:
    sourceName: user
    featureId: country
- name: operating_system
  fetchFrom:
    sourceName: user
    featureId: operating_system
- name: language
  fetchFrom:
    sourceName: user
    featureId: language
- name: cnt_user_engagement
  fetchFrom:
    sourceName: behavior
    featureId: cnt_user_engagement
- name: cnt_level_start_quickplay
  fetchFrom:
    sourceName: behavior
    featureId: cnt_level_start_quickplay
- name: cnt_level_end_quickplay
  fetchFrom:
    sourceName: behavior
    featureId: cnt_level_end_quickplay
- name: cnt_level_complete_quickplay
  fetchFrom:
    sourceName: behavior
    featureId: cnt_level_complete_quickplay
- name: cnt_level_reset_quickplay
  fetchFrom:
    sourceName: behavior
    featureId: cnt_level_reset_quickplay
- name: cnt_post_score
  fetchFrom:
    sourceName: behavior
    featureId: cnt_post_score
- name: cnt_spend_virtual_currency
  fetchFrom:
    sourceName: behavior
    featureId: cnt_spend_virtual_currency
- name: cnt_ad_reward
  fetchFrom:
    sourceName: behavior
    featureId: cnt_ad_reward
- name: cnt_challenge_a_friend
  fetchFrom:
    sourceName: behavior
    featureId: cnt_challenge_a_friend
- name: cnt_completed_5_levels
  fetchFrom:
    sourceName: behavior
    featureId: cnt_completed_5_levels
- name: cnt_use_extra_steps
  fetchFrom:
    sourceName: behavior
    featureId: cnt_use_extra_steps
- name: month
  fetchFrom:
    sourceName: behavior
    featureId: month
- name: julianday
  fetchFrom:
    sourceName: behavior
    featureId: julianday
- name: dayofweek
  fetchFrom:
    sourceName: behavior
    featureId: dayofweek"""

feature_fetch_config = FEATURE_FETCH_CONFIG_TEMPLATE.format(
    PROJECT_NUMBER=PROJECT_NUMBER, REGION=REGION, FEATURESTORE_ID=FEATURESTORE_ID
)

with open("prediction_featurestore_fetch_config.yaml", "w") as f:
    f.write(feature_fetch_config)

Copy the feature fetch config yaml to the artifacts gcs path.

In [ ]:
!gsutil cp prediction_featurestore_fetch_config.yaml $ARTIFACT_URI

# Upload and Deploy Model to Vertex Prediction

## Upload Model

In [ ]:
DISPLAY_NAME = "mobile_gaming_featureStore_integration_" + TIMESTAMP

model = aiplatform.Model.upload(
    display_name=DISPLAY_NAME,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest",
    sync=False,
)

model.wait()

## Online Prediction

### Deploy Model

#### Prediction service account

We need a provide service account for this new feature because the prediction workload's default identity does not have access to Feature Store. The service account needs to have `Vertex AI Feature Store Data Viewer` in your project.

For this bug bash, a service account has already been created for you.

In [ ]:
SA_NAME = "prediction-feature-store-fetch"
SA_DESCRIPTION = '"Fetch feature from Feature Store during Prediction"'
DISPLAY_NAME = "prediction-feature-store-fetch"

!gcloud iam service-accounts create $SA_NAME \
    --description=$SA_DESCRIPTION \
    --display-name=$DISPLAY_NAME

In [ ]:
SERVICE_ACCOUNT = f"{SA_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"

Grant the Service Account Vertex AI Feature Store Data Viewer role to access data in Feature Store during prediction.

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT \
    --role=roles/aiplatform.featurestoreDataViewer;

Now we're ready to deploy the model.

In [ ]:
DEPLOYED_NAME = DISPLAY_NAME + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MACHINE_TYPE = "n1-standard-4"

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=MACHINE_TYPE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    service_account=SERVICE_ACCOUNT,
)

### Predict

The TF model expects inputs of the following format:

In [ ]:
default_pred_request = [
    {
        "user_pseudo_id": "AB0F2EE5F9F401763BE1E9FA55410312",
        "country": "Australia",
        "operating_system": "IOS",
        "language": "en-au",
        "cnt_user_engagement": 3.0,
        "cnt_level_start_quickplay": 1.0,
        "cnt_level_end_quickplay": 0.0,
        "cnt_level_complete_quickplay": 0.0,
        "cnt_level_reset_quickplay": 0.0,
        "cnt_post_score": 0.0,
        "cnt_spend_virtual_currency": 0.0,
        "cnt_ad_reward": 0.0,
        "cnt_challenge_a_friend": 0.0,
        "cnt_completed_5_levels": 0.0,
        "cnt_use_extra_steps": 0.0,
        "month": 7,
        "julianday": 194,
        "dayofweek": 6,
    },
]

Instead of providing the feature values directly in the predict request, we'll instead specify the entity id key (e.g.`demographic` and `behavior`) and entity id value (e.g. `AB0F2EE5F9F401763BE1E9FA55410312`). The feature values will be fetched service side at predict time.

In [ ]:
fs_pred_request = [
    {
        "user_pseudo_id": "AB0F2EE5F9F401763BE1E9FA55410312",
        "demographic": "AB0F2EE5F9F401763BE1E9FA55410312",
        "behavior": "AB0F2EE5F9F401763BE1E9FA55410312",
    },
]

endpoint.predict(fs_pred_request)

Any subset of features fetched from Feature Store can be overriden. For example,

In [ ]:
fs_pred_request_with_overridden_features = [
    {
        "user_pseudo_id": "AB0F2EE5F9F401763BE1E9FA55410312",
        "demographic": "AB0F2EE5F9F401763BE1E9FA55410312",
        "behavior": "AB0F2EE5F9F401763BE1E9FA55410312",
        "cnt_ad_reward": 10.0,
        "cnt_challenge_a_friend": 10.0,
        "cnt_completed_5_levels": 10.0,
        "cnt_use_extra_steps": 10.0,
    },
]

endpoint.predict(fs_pred_request_with_overridden_features)

If all fetched features are overriden, then the behavior of the predict request is the same as that without feature store integration. Compare the responses of `fs_pred_request` and `default_pred_request`.

In [ ]:
endpoint.predict(default_pred_request)

# Debugging

We have provided a Identity model server, built using [CPR](https://cloud.google.com/vertex-ai/docs/predictions/custom-prediction-routines) to help with debugging your feature fetch config. The CPR built Identity model server simply returns the request body as the response. This will allow you to explore how the Feature Store/Prediction integration formats the prediction request to the model server.

The server has been published here - us-central1-docker.pkg.dev/fs-pred-bugbash-1/custom-container-prediction-sdk/cpr-identity, and uses the following `Predictor`.
```
from google.cloud.aiplatform.prediction.predictor import Predictor

class CprPredictor(Predictor):
    
    def __init__(self):
        return
    
    def load(self, artifacts_uri: str):
        """Loads the preprocessor and model artifacts."""
        pass

    def predict(self, instances):
        """Performs prediction."""
        return {"predictions": instances}
```

Using the same feature fetch config file as in the previous example, let's now use it with the Identity model server to debug it.

In [ ]:
IDENTITY_MODEL_DISPLAY_NAME = "cpr_identity_with_feature_store_integration" + TIMESTAMP

identity_model = aiplatform.Model.upload(
    display_name=IDENTITY_MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri="us-central1-docker.pkg.dev/fs-pred-bugbash-1/custom-container-prediction-sdk/cpr-identity",
    sync=False,
)

identity_model.wait()

In [ ]:
identity_endpoint = identity_model.deploy(
    deployed_model_display_name=IDENTITY_MODEL_DISPLAY_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=MACHINE_TYPE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    service_account=SERVICE_ACCOUNT,
)

Let's now see how the actual request to the model server looked for each of the predict requests in the previous example.

In [ ]:
print("Predict request:")
print(fs_pred_request)
print("Request sent to model server:")
identity_endpoint.predict(fs_pred_request)

In [ ]:
print("Predict request:")
print(fs_pred_request_with_overridden_features)
print("Request sent to model server:")
identity_endpoint.predict(fs_pred_request_with_overridden_features)

In [ ]:
print("Predict request:")
print(default_pred_request)
print("Request sent to model server:")
identity_endpoint.predict(default_pred_request)

# Clean Up

In [ ]:
# delete feature store
mobile_gaming_feature_store.delete(sync=True, force=True)

In [ ]:
# delete Vertex AI resources
endpoint.undeploy_all()
endpoint.delete()
model.delete

identity_endpoint.undeploy_all()
identity_endpoint.delete()
identity_model.delete

In [ ]:
# Delete bucket
!gsutil -m rm -r $BUCKET_URI